<a href="https://colab.research.google.com/github/ouafighizlene21/classification_tid2013_cnn.ipynb/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📦 Importer les bibliothèques nécessaires / Import Required Libraries  
Importer les bibliothèques essentielles pour la classification d’images avec un modèle CNN,  
y compris le traitement des images, la construction du modèle, l'entraînement, l'encodage des labels et l'affichage des résultats.  
Import essential libraries for image classification using a CNN model,  
including image processing, model building, training, label encoding, and result visualization.


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os

## 🔗 Monter Google Drive / Mount Google Drive  
Connecter Google Colab à Google Drive pour accéder aux fichiers stockés.  
Connect Google Colab to Google Drive to access stored files.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 🗂️ Définir les chemins d’accès aux images / Set Image Paths  
Définir les chemins des images de référence et des images distordues stockées dans Google Drive.  
Set the paths to reference and distorted images stored in Google Drive.


In [3]:
# Définir le chemin vers les images de réfrence
reference_dir = '/content/drive/My Drive/TID2013/reference_images'
# Définir le chemin vers les images de réfrence
distorted_dir = '/content/drive/My Drive/TID2013/distorted_images'

## 🖼️ Charger et préparer les images / Load and Prepare Images  
Charger les images de référence et distordues depuis leurs répertoires respectifs, les redimensionner à 224x224 pixels,  
les convertir en tableaux, les normaliser, puis attribuer une étiquette à chaque image.  
Load reference and distorted images from their respective directories, resize them to 224x224 pixels,  
convert them to arrays, normalize them, and assign a label to each image.


In [4]:
def load_tid2013(data_dir, label):
    images = []
    labels = []
    for img_file in os.listdir(data_dir):
        img_path = os.path.join(data_dir, img_file)
        img = image.load_img(img_path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = img / 255.0
        images.append(img)
        labels.append(label)
    return images, labels

ref_images, ref_labels = load_tid2013(reference_dir, 'reference')
dist_images, dist_labels = load_tid2013(distorted_dir, 'distorted')

# Combiner les données des deux répertoires
images = np.array(ref_images + dist_images)
labels = np.array(ref_labels + dist_labels)

## ✂️ Diviser les données en ensembles d’entraînement et de test / Split Data into Training and Testing Sets  
Diviser les images et leurs étiquettes en un ensemble d’entraînement (80%) et un ensemble de test (20%) pour évaluer les performances du modèle.  
Split the images and their labels into a training set (80%) and a testing set (20%) to evaluate model performance.


In [5]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

## 🔤 Encoder les étiquettes / Encode Labels  
Utiliser `LabelEncoder` pour convertir les étiquettes textuelles (comme "reference" et "distorted") en valeurs numériques.  
`fit_transform` est appliqué sur les données d'entraînement pour ajuster et transformer, tandis que `transform` est utilisé sur les données de test.  
Use `LabelEncoder` to convert textual labels (e.g., "reference" and "distorted") into numeric values.  
Apply `fit_transform` on the training set to fit and transform simultaneously, and use `transform` on the test set to apply the learned mapping.


In [7]:
#utilisez fit_transform sur les données d'entraînement pour ajuster et transformer simultanément, et utilisez transform sur les données de test ou toute nouvelle donnée pour appliquer la transformation apprise.
from sklearn.preprocessing import LabelEncoder
# Initialiser l'encodeur
le = LabelEncoder()

# Encoder les étiquettes d'entraînement
y_train_encoded = le.fit_transform(y_train)

# Encoder les étiquettes de test
y_test_encoded = le.transform(y_test)

# Afficher les classes apprises par l'encodeur
print("Classes apprises par l'encodeur :", le.classes_)

Classes apprises par l'encodeur : ['distorted' 'reference']


## 🧠 Construire et compiler le modèle CNN / Build and Compile the CNN Model  
Encoder les étiquettes en vecteurs one-hot pour la classification multiclasse.  
Construire un modèle CNN avec des couches de convolution, de max-pooling, une couche flatten,  
et des couches denses entièrement connectées. Compiler ensuite le modèle avec l'optimiseur Adam  
et la fonction de perte `categorical_crossentropy`.  
Encode the labels using one-hot encoding for multi-class classification.  
Build a CNN model with convolutional layers, max-pooling, a flatten layer,  
and fully connected dense layers. Then compile the model using the Adam optimizer  
and the `categorical_crossentropy` loss function.


In [13]:
# Encodage one-hot des labels
y_train_one_hot = to_categorical(y_train_encoded, num_classes=2)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=2)

# Création du modèle
model = Sequential()

# Convolution + MaxPooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Aplatir les sorties de la dernière couche de convolution
model.add(Flatten())

# Couches entièrement connectées
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 neurones avec softmax pour multi-classes

# Compilation du modèle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,218 (42.61 MB)

 Trainable params: 11,169,218 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

## 🚀 Entraîner le modèle CNN / Train the CNN Model  
Entraîner le modèle sur l’ensemble d'entraînement pendant 10 époques avec une taille de lot de 32.  
Les performances sont validées à chaque époque sur l’ensemble de test.  
Train the model on the training set for 10 epochs with a batch size of 32.  
Performance is validated at each epoch on the test set.


In [15]:
# Entraînement du modèle
history = model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

Epoch 1/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 267s 4s/step - accuracy: 0.9877 - loss: 0.0991 - val_accuracy: 0.9934 - val_loss: 0.0448
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 266s 4s/step - accuracy: 0.9918 - loss: 0.0545 - val_accuracy: 0.9934 - val_loss: 0.0413
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 323s 4s/step - accuracy: 0.9892 - loss: 0.0676 - val_accuracy: 0.9934 - val_loss: 0.0483
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.9911 - loss: 0.0613 - val_accuracy: 0.9934 - val_loss: 0.0451
Epoch 5/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 326s 3s/step - accuracy: 0.9931 - loss: 0.0475 - val_accuracy: 0.9934 - val_loss: 0.0528
Epoch 6/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.9906 - loss: 0.0591 - val_accuracy: 0.9934 - val_loss: 0.0433
Epoch 7/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 321s 3s/step - accuracy: 0.9915 - loss: 0.0514 - val_accuracy: 0.9934 - val_loss: 0.0419
Epoch 8/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 323s 3s/step - accuracy: 0.9930 - loss: 0.0427 - val_accuracy: 0.9934 - v

## 💾 Sauvegarder le modèle et l’historique d'entraînement / Save the Model and Training History  
Sauvegarder le modèle entraîné au format `.keras` pour pouvoir le recharger ultérieurement sans le réentraîner.  
Enregistrer également l'historique d'entraînement (précision, perte, etc.) dans un fichier `.pkl`  
pour pouvoir visualiser les courbes plus tard.  
Save the trained model in `.keras` format to reload it later without retraining.  
Also save the training history (accuracy, loss, etc.) in a `.pkl` file to visualize the curves later.


In [16]:
import pickle

# Sauvegarder l'historique d'entraînement
with open('historique_entraînement.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Sauvegarde du modèle dans le nouveau format .keras
model.save('mon_modele_entraine.keras')


## 🔄 Recharger l’historique d’entraînement / Reload Training History  
Charger les métriques d’entraînement précédemment sauvegardées (précision, perte, etc.)  
depuis un fichier `.pkl` pour permettre leur visualisation sans relancer l'entraînement.  
Load previously saved training metrics (accuracy, loss, etc.)  
from a `.pkl` file to allow visualization without retraining.


In [18]:
# Recharger l'historique
with open('historique_entraînement.pkl', 'rb') as f:
    history_dict = pickle.load(f)


## 📂 Charger le modèle sauvegardé / Load the Saved Model  
Recharger le modèle CNN sauvegardé au format `.keras` afin de l’utiliser pour l’évaluation ou les prédictions,  
sans avoir à le réentraîner.  
Reload the CNN model saved in `.keras` format to use it for evaluation or predictions,  
without retraining it.


In [19]:
# Charger le modèle sauvegardé
model = load_model('mon_modele_entraine.keras')


## 🖼️ Préparer une image pour la prédiction / Prepare an Image for Prediction  
Définir une fonction pour charger une image, la redimensionner, la convertir en tableau numpy,  
la normaliser et lui ajouter une dimension pour l’adapter à l’entrée du modèle.  
Define a function to load an image, resize it, convert it to a NumPy array,  
normalize it, and add a batch dimension to fit the model input.


In [20]:
# Fonction pour préparer l'image
def prepare_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)  # Charger l'image
    img_array = image.img_to_array(img)  # Convertir en tableau numpy
    img_array = img_array / 255.0  # Normaliser
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le batch
    return img_array


## 🏷️ Afficher les classes apprises par l’encodeur / Show Classes Learned by the Encoder  
Afficher l’ordre des classes (étiquettes) que le `LabelEncoder` a appris lors de l’encodage des données.  
Cela permet de connaître la correspondance exacte entre les indices numériques (0, 1, ...) et les étiquettes textuelles (`reference`, `distorted`, etc.).  
Display the order of class labels learned by the `LabelEncoder` when encoding data.  
This ensures correct mapping between numeric indices (0, 1, ...) and textual labels (`reference`, `distorted`, etc.).


In [35]:
print("Classes connues par l'encodeur :", le.classes_)


Classes connues par l'encodeur : ['distorted' 'reference']


## 🔎 Prédire la classe d’une image avec le modèle entraîné / Predict the Class of an Image Using the Trained Model  
Charger une image issue du dossier des images distordues, la préparer, et l’utiliser comme entrée pour le modèle CNN.  
La prédiction est ensuite interprétée en utilisant les classes apprises automatiquement par le `LabelEncoder`,  
ce qui garantit la cohérence entre l’index prédit et le nom de la classe (`distorted`, `reference`, etc.).  
Load a distorted image, preprocess it, and use it as input to the trained CNN model.  
The predicted class index is mapped back to the actual class name using the labels learned by the `LabelEncoder`,  
ensuring consistency between prediction index and true label (`distorted`, `reference`, etc.).


In [41]:
# Préparer l'image pour la prédiction
img_path = '/content/drive/My Drive/TID2013/distorted_images/i01_18_4.jpg'
prepared_image = prepare_image(img_path)

# Faire la prédiction
predictions = model.predict(prepared_image)

# Utiliser directement les classes de l'encodeur
predicted_label = le.classes_[np.argmax(predictions)]

print(f"Classe prédite : {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Classe prédite : distorted


## 🧠 Prédire la classe d’une image avec le modèle entraîné / Predict the Class of an Image Using the Trained Model  
Charger une image issue du dossier des images distordues, la préparer, et l’utiliser comme entrée pour le modèle CNN.  
La prédiction est ensuite interprétée en utilisant les classes apprises automatiquement par le `LabelEncoder`,  
ce qui garantit la cohérence entre l’index prédit et le nom de la classe (`distorted`, `reference`, etc.).  
Load a distorted image, preprocess it, and use it as input to the trained CNN model.  
The predicted class index is mapped back to the actual class name using the labels learned by the `LabelEncoder`,  
ensuring consistency between prediction index and true label (`distorted`, `reference`, etc.).

⚠️ **Remarque importante / Important Note:**  
L’image utilisée pour ce test provient du **dossier `reference_images`**,  
mais le modèle a prédit à tort qu’elle appartient à la classe `"distorted"`.  
Cela montre que le modèle peut faire des erreurs, probablement à cause du fort déséquilibre entre les classes lors de l'entraînement.  
The image used for this test comes from the **`reference_images` folder**,  
but the model mistakenly predicted it as belonging to the `"distorted"` class.  
This highlights that the model can make mistakes, likely due to strong class imbalance during training.


In [44]:
# Préparer l'image pour la prédiction
img_path = '/content/drive/My Drive/TID2013/reference_images/I11.jpg'  # Chemin de l'image
prepared_image = prepare_image(img_path)

# Faire la prédiction
predictions = model.predict(prepared_image)

# Utiliser le même ordre de classes que l'encodeur
class_labels = list(le.classes_)
predicted_class = np.argmax(predictions, axis=1)
predicted_label = class_labels[predicted_class[0]]

print(f"Classe prédite : {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Classe prédite : distorted


## ⚠️ Attention : correspondance des classes et prédictions  
## ⚠️ Warning: Class Mapping and Predictions

Lorsque vous utilisez `np.argmax(predictions)` pour obtenir l’indice de la classe prédite,  
il est **essentiel d’utiliser les vraies étiquettes apprises par le `LabelEncoder`**,  
et **pas une liste manuelle comme `['reference', 'distorted']`**.

When using `np.argmax(predictions)` to get the predicted class index,  
it is **crucial to use the actual class labels learned by the `LabelEncoder`**,  
and **not a manually defined list like `['reference', 'distorted']`**.

---

### 💡 Mauvais exemple / Bad example:
```python
class_labels = ['reference', 'distorted']
predicted_label = class_labels[np.argmax(predictions)]


In [45]:
# Préparer l'image pour la prédiction
img_path = '/content/drive/My Drive/TID2013/reference_images/I11.jpg'  # Chemin de l'image
prepared_image = prepare_image(img_path)

# Faire la prédiction
predictions = model.predict(prepared_image)

# Utiliser le même ordre de classes que l'encodeur
predicted_class = np.argmax(predictions, axis=1)
class_labels = ['reference', 'distorted']  # Les étiquettes de tes classes
predicted_label = class_labels[predicted_class[0]]

print(f"Classe prédite : {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Classe prédite : reference
